# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import time
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:


# File to Load
file_to_load = "../WeatherPy/output_data/cities.csv"

# Read Purchasing File and store into Pandas data frame
city_df = pd.read_csv(file_to_load)
city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapaa,22.0752,-159.3190,82.38,82,64,1.99,US,1627102515
1,Makurdi,7.7411,8.5121,71.64,98,100,4.12,NG,1627102757
2,Ciudad Guayana,8.3533,-62.6528,73.94,96,91,3.09,VE,1627102758
3,Beyneu,45.3167,55.2000,87.87,37,5,10.02,KZ,1627102758
4,Hermanus,-34.4187,19.2345,47.89,60,0,9.48,ZA,1627102758
...,...,...,...,...,...,...,...,...,...
705,Mīāndoāb,36.9694,46.1027,83.53,36,0,2.98,IR,1627102992
706,Lorengau,-2.0226,147.2712,83.80,78,95,9.80,PG,1627102993
707,Ryazhsk,53.7038,40.1109,60.55,66,69,5.19,RU,1627102993
708,Evensk,61.9500,159.2333,50.58,96,100,2.39,RU,1627102960


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [4]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df["Humidity"]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,max_intensity = 100,point_radius = 5)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df = city_df[(city_df['Max Temp'] > 70) & (city_df['Max Temp'] < 90)]
ideal_df = ideal_df[ideal_df['Wind Speed'] < 10]
ideal_df = ideal_df[ideal_df['Cloudiness'] < 5]
ideal_df = ideal_df[ideal_df['Humidity'] < 60]
hotel_df = ideal_df
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,Saint George,37.1041,-113.5841,86.63,45,1,1.01,US,1627102550
103,Muş Province,39.0000,41.7500,70.79,46,0,6.80,TR,1627102516
109,Ürümqi,43.8010,87.6005,86.95,27,0,6.71,CN,1627102698
203,Awjilah,29.1081,21.2869,76.89,47,0,9.10,LY,1627102703
295,Portland,45.5234,-122.6762,76.44,57,0,1.01,US,1627102387
301,Ada,43.4166,-116.1262,76.98,30,0,5.01,US,1627102865
315,Namangan,40.9983,71.6726,86.45,39,0,4.61,UZ,1627102869
340,Mount Isa,-20.7333,139.5000,80.37,21,4,8.05,AU,1627102878
351,Saint-Pierre,-21.3393,55.4781,73.29,50,0,5.75,RE,1627102881
436,Sterlibashevo,53.4376,55.2578,72.64,41,1,5.17,RU,1627102907


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df['Hotel Name'] = ""
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,Saint George,37.1041,-113.5841,86.63,45,1,1.01,US,1627102550,
103,Muş Province,39.0000,41.7500,70.79,46,0,6.80,TR,1627102516,
109,Ürümqi,43.8010,87.6005,86.95,27,0,6.71,CN,1627102698,
203,Awjilah,29.1081,21.2869,76.89,47,0,9.10,LY,1627102703,
295,Portland,45.5234,-122.6762,76.44,57,0,1.01,US,1627102387,
301,Ada,43.4166,-116.1262,76.98,30,0,5.01,US,1627102865,
315,Namangan,40.9983,71.6726,86.45,39,0,4.61,UZ,1627102869,
340,Mount Isa,-20.7333,139.5000,80.37,21,4,8.05,AU,1627102878,
351,Saint-Pierre,-21.3393,55.4781,73.29,50,0,5.75,RE,1627102881,
436,Sterlibashevo,53.4376,55.2578,72.64,41,1,5.17,RU,1627102907,


In [7]:
for index, row in hotel_df.iterrows():
    try:
        
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        }


        lat = row['Lat']
        lng = row['Lng']

        params['location'] = f"{lat}, {lng}"

        hotel_data = requests.get(base_url, params=params).json()
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except IndexError:
        
        hotel_df.loc[index, "Hotel Name"] = "NaN"
        
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,Saint George,37.1041,-113.5841,86.63,45,1,1.01,US,1627102550,"The Advenire, Autograph Collection"
103,Muş Province,39.0000,41.7500,70.79,46,0,6.80,TR,1627102516,NaN
109,Ürümqi,43.8010,87.6005,86.95,27,0,6.71,CN,1627102698,Sheraton Urumqi Hotel
203,Awjilah,29.1081,21.2869,76.89,47,0,9.10,LY,1627102703,NaN
295,Portland,45.5234,-122.6762,76.44,57,0,1.01,US,1627102387,Tiny Digs Hotel Portland
301,Ada,43.4166,-116.1262,76.98,30,0,5.01,US,1627102865,NaN
315,Namangan,40.9983,71.6726,86.45,39,0,4.61,UZ,1627102869,Shedevr Plaza
340,Mount Isa,-20.7333,139.5000,80.37,21,4,8.05,AU,1627102878,Isa Hotel
351,Saint-Pierre,-21.3393,55.4781,73.29,50,0,5.75,RE,1627102881,Hôtel Le Saint-Pierre
436,Sterlibashevo,53.4376,55.2578,72.64,41,1,5.17,RU,1627102907,Zur Eni


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))